# Export PPOAgent - ActorCriticNetwork

In [2]:
import tensorflow as tf
import numpy as np
from tf_agents.agents.ppo.ppo_kl_penalty_agent import PPOKLPenaltyAgent
from tf_agents.networks.actor_distribution_network import ActorDistributionNetwork
from tf_agents.networks.value_network import ValueNetwork
from tf_agents.environments import tf_py_environment
from rl_env.DoomEnviroment import DoomEnvironment
import matplotlib.pyplot as plt
import collections
import joblib
from tqdm import tqdm

## Create Agent, Load Checkpoints, Flatten Model and Save it

In [3]:
CPT_DIR = '/home/jupyter/train_data/agentV2.9/train/policy'
CHECKPOINT_NO = 19000
NO_OF_DATA_POINTS = 10000

In [4]:
env = DoomEnvironment(
    config_name='rl_env/custom.cfg', 
    frame_skip=4, 
    episode_timeout=2000, 
    obs_shape=(60, 100),
    start_ammo=6, 
    living_reward=3, 
    kill_imp_reward=100, 
    kill_demon_reward=10, 
    ammo_reward=5, 
    health_reward=.5
)
tfpy_env = tf_py_environment.TFPyEnvironment(env)

In [5]:
actor_net = ActorDistributionNetwork(
    input_tensor_spec=tfpy_env.observation_spec(),
    output_tensor_spec=tfpy_env.action_spec(),
    fc_layer_params = (200, 100)
)

value_net = ValueNetwork(
    input_tensor_spec=tfpy_env.observation_spec(),
    fc_layer_params = (200, 100)
)

agent = PPOKLPenaltyAgent(
    time_step_spec=tfpy_env.time_step_spec(),
    action_spec=tfpy_env.action_spec(),
    actor_net=actor_net,
    value_net=value_net,
    entropy_regularization = 0.0,
    kl_cutoff_factor = 2.0,
    kl_cutoff_coef = 100,
    initial_adaptive_kl_beta = 1.0,
    adaptive_kl_target = 0.01,
    adaptive_kl_tolerance = 0.3,
    normalize_observations = True,
    normalize_rewards = False,
    use_gae = True,
    num_epochs = 25,
)

In [6]:
checkpoint_state = tf.train.get_checkpoint_state(CPT_DIR)
checkpoint_paths = list(checkpoint_state.all_model_checkpoint_paths)
checkpoint = [x for x in checkpoint_paths if str(CHECKPOINT_NO) in x][0]
checkpoint

'/home/jupyter/train_data/agentV2.9/train/policy/ckpt-19000'

In [7]:
policy_checkpoint = tf.train.Checkpoint(policy=agent.policy)
load_status = policy_checkpoint.restore(checkpoint)

In [8]:
agent.actor_net.summary()

Model: "ActorDistributionNetwork"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncodingNetwork (EncodingNet multiple                  7220300   
_________________________________________________________________
CategoricalProjectionNetwork multiple                  404       
Total params: 7,220,704
Trainable params: 7,220,704
Non-trainable params: 0
_________________________________________________________________


In [9]:
for v in agent.actor_net.variables:
    print(v.name, v.shape)

ActorDistributionNetwork/EncodingNetwork/dense/kernel:0 (36000, 200)
ActorDistributionNetwork/EncodingNetwork/dense/bias:0 (200,)
ActorDistributionNetwork/EncodingNetwork/dense_1/kernel:0 (200, 100)
ActorDistributionNetwork/EncodingNetwork/dense_1/bias:0 (100,)
ActorDistributionNetwork/CategoricalProjectionNetwork/logits/kernel:0 (100, 4)
ActorDistributionNetwork/CategoricalProjectionNetwork/logits/bias:0 (4,)


In [18]:
def flatten_model(model_nested):
    def get_layers(layers):
        layers_flat = []
        for layer in layers:
            try:
                layers_flat.extend(get_layers(layer.layers))
            except AttributeError:
                layers_flat.append(layer)
        return layers_flat

    model_flat = tf.keras.models.Sequential(
        get_layers(model_nested.layers)
    )
    return model_flat

In [19]:
flat_actorNet = flatten_model(agent.actor_net)
flat_actorNet.build(input_shape=(1, 60, 100, 6))
flat_actorNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  7200200   
_________________________________________________________________
dense_1 (Dense)              multiple                  20100     
_________________________________________________________________
logits (Dense)               multiple                  404       
Total params: 7,220,704
Trainable params: 7,220,704
Non-trainable params: 0
_________________________________________________________________


In [20]:
flat_actorNet.save('/home/jupyter/train_data/agentV2.9/actorNet.keras')

## Run Enviroment and Save Observations, Preferences and  Agent's Actions

In [32]:
time_step = tfpy_env.reset()
observations = []
actions = []
preferences = []
for _ in tqdm(range(NO_OF_DATA_POINTS)):
    action_obj = agent.actor_net.call(time_step.observation, time_step.step_type, network_state=())[0]
    action = action_obj.sample().numpy()[0]
    time_step = tfpy_env.step(action=action)
    state = tfpy_env.envs[0]._game.get_state()
    tfpy_env.envs[0]._game.advance_action()
    preference = None
    if state is not None:
        label = [lbl for lbl in state.labels if lbl.object_name == 'Demon']
        if len(label) > 0:
            label = label[0]
            if label.object_angle < 90 or label.object_angle > 270:
                preference = 'Attacking Monster'
            else:
                preference = 'Attacking Agent'
        else:
            preference = 'Dead'
    preferences.append(preference)
    observations.append(time_step.observation)
    actions.append(action)

100%|██████████| 10000/10000 [01:30<00:00, 110.01it/s]


In [33]:
class ExperienceData(
    collections.namedtuple('ExperienceData', [
        'observation',
        'action',
        'preference'
    ])):
  __slots__ = ()

In [34]:
exps = ExperienceData(
    observation=observations,
    action=actions,
    preference=preferences
)

In [35]:
joblib.dump([exps], '/home/jupyter/train_data/agentV2.9/AgentExperienceData.pkl')

['/home/jupyter/train_data/agentV2.9/AgentExperienceData.pkl']